#  Training YOLO on Fire Detection for Satellite Scenario

## Overview
In this notebook, we train a **YOLOv8** model on a custom dataset designed for detecting **fire scenarios** in **satellite images**. This dataset represents one of the specific scenarios for our **Mixture of Experts (MoE)** model, where each expert specializes in a different scenario (e.g., fire detection in indoor, indoor, satellite, or far-field environments).

### Key Steps:
- **Dataset**: The model is trained using a **satellite fire detection dataset**, which consists of images with fire-related features captured from satellite imagery.
- **YOLOv8 Training**: The YOLOv8 model is fine-tuned on this dataset, learning to identify fire-related objects.
- **Scenario Expert**: This trained model acts as an expert specifically for detecting fire in satellite images and is part of a broader MoE-based approach for multi-scenario detection.



##  Data Loading & Preprocessing


In [1]:
import os
import random
import shutil

# Set seed for reproducibility
random.seed(42)

# Paths
original_images_dir = "train/images"
original_labels_dir = "train/labels"

train_subset_images_dir = "satellite_fire_dataset/images/train_subset"
train_subset_labels_dir = "satellite_fire_dataset/labels/train_subset"

test_subset_images_dir = "satellite_fire_dataset/images/test_subset"
test_subset_labels_dir = "satellite_fire_dataset/labels/test_subset"

# Create directories if they don't exist
os.makedirs(train_subset_images_dir, exist_ok=True)
os.makedirs(train_subset_labels_dir, exist_ok=True)
os.makedirs(test_subset_images_dir, exist_ok=True)
os.makedirs(test_subset_labels_dir, exist_ok=True)

# List all image files
all_images = [f for f in os.listdir(original_images_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Randomly select 3100 images total
sampled_images = random.sample(all_images, 3100)

# Split into 3000 train, 100 test
train_images = sampled_images[:3000]
test_images = sampled_images[3000:]

# Helper function to copy images and labels
def copy_data(img_list, img_dest, label_dest):
    for img_file in img_list:
        # Copy image
        shutil.copy(
            os.path.join(original_images_dir, img_file),
            os.path.join(img_dest, img_file)
        )

        # Get corresponding label file
        label_file = os.path.splitext(img_file)[0] + ".txt"
        original_label_path = os.path.join(original_labels_dir, label_file)

        # Copy label if it exists
        if os.path.exists(original_label_path):
            shutil.copy(original_label_path, os.path.join(label_dest, label_file))

# Copy training data
copy_data(train_images, train_subset_images_dir, train_subset_labels_dir)

# Copy test data
copy_data(test_images, test_subset_images_dir, test_subset_labels_dir)

print("✅ Sampled 3000 training and 100 test images and labels from original training set.")


✅ Sampled 3000 training and 100 test images and labels from original training set.


##  Training YOLOv8 on Satellite Fire Detection

We utilize the **YOLOv8** object detection model to train on a custom dataset defined in `satellite.yaml`. The configuration for training is as follows:

- 🧠 **Model**: `yolov8n.pt` (Nano variant — optimized for speed and prototyping)
- 📁 **Dataset**: Defined in `satellite.yaml`
- 🖼️ **Image Size**: 640×640
- 🔁 **Epochs**: 100
- 📦 **Batch Size**: 16
- 💻 **Device**: GPU (device 0)




In [6]:
import torch
print("GPU available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0))

GPU available: True
Current device: NVIDIA GeForce RTX 3070


In [7]:
from ultralytics import YOLO

# Load a YOLOv8 model (choose from yolov8n, yolov8s, yolov8m, etc.)
model = YOLO("yolov8n.pt")  # you can replace with 'yolov8s.pt' for better accuracy

# Train the model
model.train(
    data="satellite.yaml",     # Path to your custom dataset YAML
    epochs=100,                # Number of training epochs
    imgsz=640,                 # Image size
    batch=16,                  # Batch size (adjust based on your GPU memory)
    device = 0
)


Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=satellite.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=No

train: Scanning C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\labels\train_subset... 3000 images, 60 backgrounds, 0 corrupt: 100%|██████████| 3000/3000 [00:04<00:00, 740.73it/s]


train: New cache created: C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\labels\train_subset.cache


val: Scanning C:\Users\pc\Desktop\DL\Project\Satellite\valid\labels... 200 images, 1 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<00:00, 1446.28it/s]

val: New cache created: C:\Users\pc\Desktop\DL\Project\Satellite\valid\labels.cache


...

                   all        200        317      0.446      0.441      0.379      0.214



100 epochs completed in 0.546 hours.
Optimizer stripped from runs\detect\train\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train\weights\best.pt, 6.3MB

Validating runs\detect\train\weights\best.pt...
Ultralytics 8.3.107  Python-3.11.6 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8192MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.36it/s]


                   all        200        317      0.517      0.448      0.447      0.253
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001E389B82350>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

##  Inference on Satellite Test Images using Trained YOLOv8 Model

We perform inference using the **best trained YOLOv8 model** to evaluate its performance on the **test set** of satellite fire scenarios.

- 🎯 **Model Weights**: `best.pt` from the training run
- 💾 **Output**: Predictions will be saved automatically by YOLO

This step visually validates how well the model generalizes to unseen indoor fire detection cases.


In [8]:
from ultralytics import YOLO

# Load your trained weights
model = YOLO("runs/detect/train/weights/best.pt")


In [11]:
results = model.predict(source="C:/Users/pc/Desktop/DL/Project/Satellite/satellite_fire_dataset/images/test_subset", save=True)



image 1/100 C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\images\test_subset\cl6b0msow001cl4559m4m56le_2_FALSE_COLOR_jpg.rf.f68ad2780e71b8384276d2b5e9f13de2.jpg: 640x640 (no detections), 4.9ms
image 2/100 C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\images\test_subset\cl6b0skkx001nl4557zd344qi_6_SWIR_jpg.rf.050130f50407894cc5e0827db1da6078.jpg: 640x640 1 fire, 6.4ms
image 3/100 C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\images\test_subset\cl6b4x9pw001pl455en07csnz_1_TRUE_COLOR_jpg.rf.13266e9caca2d71bea0935e8608a6342.jpg: 640x640 1 fire, 4.2ms
image 4/100 C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\images\test_subset\cl6b55k98002fl455fxpwaq8j_1_TRUE_COLOR_jpg.rf.f3fc4af2e1bc1a94c52eaa7f2d2e81ce.jpg: 640x640 1 fire, 4.5ms
image 5/100 C:\Users\pc\Desktop\DL\Project\Satellite\satellite_fire_dataset\images\test_subset\cl6b5bl3w0032l4550j2k2te5_2_FALSE_COLOR_jpg.rf.f4109cccb8917845f0299ca2e1824824.jpg: 640x640 (no